# Preparation

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Feger/NER-CTI

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Feger/NER-CTI


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

In [3]:
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda-autodetect,transformers,lookups]'
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.5.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 

# Hardware Specifications


In [4]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-616858e9-5652-63ea-cf1a-e7df8574dc8e)


In [5]:
!nvidia-smi

Mon Mar  6 20:22:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    24W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz
Stepping:                        3
CPU MHz:                         2000.212
BogoMIPS:                        4000.42
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        1 MiB
L3 cache:                        38.5 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 

# Loading data assets

In [7]:
# This command fetches the data assets that should later be converted to .spacy
!spacy project assets

ℹ Fetching 3 asset(s)
✔ Downloaded asset /content/drive/MyDrive/Colab
Notebooks/Feger/NER-CTI/assets/CyNER/train.txt
✔ Downloaded asset /content/drive/MyDrive/Colab
Notebooks/Feger/NER-CTI/assets/CyNER/valid.txt
✔ Downloaded asset /content/drive/MyDrive/Colab
Notebooks/Feger/NER-CTI/assets/CyNER/test.txt


# spaCy'fy the data data

In [8]:
!spacy project run preparation

ℹ Running workflow 'preparation'

==================================== info ====================================
ℹ Skipping 'info': nothing changed

======================= brace-yourself-data-is-coming =======================
Running command: mkdir -p corpus/CyNER/
Running command: spacy convert assets/CyNER/train.txt ./corpus/CyNER/ --file-type spacy --converter conll
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (2811 documents): corpus/CyNER/train.spacy
Running command: spacy convert assets/CyNER/valid.txt ./corpus/CyNER/ --file-type spacy --converter conll
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (813 documents): corpus/CyNER/valid.spacy
Running command: spacy convert assets/CyNER/test.txt ./corpus/CyNER/ --file-type spacy --converter con

# Training and evaluation of the "classic model"


In [9]:
!spacy project run classic-model

ℹ Running workflow 'classic-model'

========================== debug-data-classic-model ==========================
Running command: spacy debug data ./configs/training_classic_model_config.cfg --verbose --paths.train ./corpus/CyNER/train.spacy --paths.dev ./corpus/CyNER/dev.spacy

============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
2811 training docs
813 evaluation docs
⚠ 12 training examples also in evaluation data

============================== Vocab & Vectors ==============================
ℹ 68191 total word(s) in the data (7954 unique)
10 most common words: 'the' (3674), '.' (2664), ',' (2597), 'to' (1648), 'of'
(1410), 'and' (1256), 'a' (1105), 'is' (787), 'in' (784), 'The' (568)
ℹ 514157 vectors (514157 unique keys, 300 dimensions)
⚠ 2944 words in training data without 

#Training and evaluation of the "foundation model"

In [10]:
!spacy project run foundation-model

ℹ Running workflow 'foundation-model'

======================== debug-data-foundation-model ========================
Running command: spacy debug data ./configs/training_foundation_model_config.cfg --verbose --paths.train ./corpus/CyNER/train.spacy --paths.dev ./corpus/CyNER/dev.spacy

============================ Data file validation ============================
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSeque